# Run helpers notebooks

In [0]:
%run "../helpers/configuration"

In [0]:
%run "../helpers/functions"

# Ingest and process raw data

In [0]:
# Define schema
from pyspark.sql.types import  StructType, StructField, IntegerType, StringType

schema = StructType(fields=[StructField("raceId", IntegerType(), False),
                            StructField("driverId", IntegerType(), True),
                            StructField("stop", StringType(), True),
                            StructField("lap", IntegerType(), True),
                            StructField("time", StringType(), True),
                            StructField("duration", StringType(), True),
                            StructField("milliseconds", IntegerType(), True)])

In [0]:
# Read json file

pitstops_df = spark.read.json(f"{raw_folder_path}/pit_stops.json", multiLine=True, schema=schema)
# display(pitstops_df)

In [0]:
# Process data
pitstops_processed = pitstops_df.withColumnRenamed("driverId", "driver_id") \
    .withColumnRenamed("raceId", "race_id")

# display(pitstops_processed)                    

In [0]:
# Add ingestion date
pitstops_final = add_ingestion_date(pitstops_processed)

In [0]:
# Write df into parquet
pitstops_final.write.parquet(f"{processed_folder_path}/pit_stops", mode="overwrite")

In [0]:
# Read data to check if it have been properly written
display(spark.read.parquet(f"{processed_folder_path}/pit_stops"))